In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import  torchvision.transforms as transforms
from torchvision import datasets, transforms,models
from torch.utils.data import DataLoader


In [40]:
class ConvBlock(nn.Module):
  def __init__(self,in_channels,out_channels,kernel_size,stride =1,padding =0):
    super().__init__()
    self.conv = nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding)
    self.relu = nn.ReLU(inplace=True)
  def forward(self,x):
    return self.relu(self.conv(x))

In [41]:
class FireModule(nn.Module):
  def __init__(self,in_channels,squeeze_channels,expand1x1_channels,expand3x3_channels):
    super().__init__()
    self.squeeze = ConvBlock(in_channels,squeeze_channels,kernel_size=1)

    self.expand1x1 = ConvBlock(squeeze_channels,expand1x1_channels,kernel_size=1)
    self.expand3x3 = ConvBlock(squeeze_channels,expand3x3_channels,kernel_size=3,padding=1)



  def forward(self,x):
    x = self.squeeze(x)
    return torch.concat([
        self.expand1x1(x),
        self.expand3x3(x)
    ],dim = 1)

In [42]:
class SqueezeNet(nn.Module):
  def __init__(self,num_classes=1000):
    super().__init__()

    self.features = nn.Sequential(
        ConvBlock(3,96,kernel_size=7,stride=2),
        nn.MaxPool2d(3,stride = 2),

        FireModule(96,16,64,64),
        FireModule(128,16,64,64),
        FireModule(128,32,128,128),
        nn.MaxPool2d(3,stride=2),

        FireModule(256,32,128,128),
        FireModule(256,48,192,192),
        FireModule(384,48,192,192),
        FireModule(384,64,256,256),
        nn.MaxPool2d(3,stride=2),

        FireModule(512,64,256,256)
    )

    self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Conv2d(512, num_classes, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )

def forward(self, x):
    x = self.features(x)
    x = self.classifier(x)
    return torch.flatten(x, 1)


In [52]:
squeeze_net = SqueezeNet(5)
print(squeeze_net)

SqueezeNet(
  (features): Sequential(
    (0): ConvBlock(
      (conv): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
      (relu): ReLU(inplace=True)
    )
    (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): FireModule(
      (squeeze): ConvBlock(
        (conv): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
        (relu): ReLU(inplace=True)
      )
      (expand1x1): ConvBlock(
        (conv): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
        (relu): ReLU(inplace=True)
      )
      (expand3x3): ConvBlock(
        (conv): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
      )
    )
    (3): FireModule(
      (squeeze): ConvBlock(
        (conv): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
        (relu): ReLU(inplace=True)
      )
      (expand1x1): ConvBlock(
        (conv): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
        (relu): ReLU(inplace=True)
      )


In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
train_dir = '/content/drive/MyDrive/Computer Vision/flowers/train'
eval_dir = '/content/drive/MyDrive/Computer Vision/flowers/val'

In [45]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

train_dataset = datasets.ImageFolder(train_dir,transform=data_transforms['train'])
eval_dataset = datasets.ImageFolder(eval_dir,transform=data_transforms['val'])

train_loader = DataLoader(train_dataset,batch_size = 16,shuffle=True)
eval_loader = DataLoader(eval_dataset,batch_size = 16)

In [46]:

from torchvision.models import SqueezeNet1_1_Weights

model = models.squeezenet1_1(weights=SqueezeNet1_1_Weights.DEFAULT)

print(model)
for param in model.parameters():
    param.requires_grad = False


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

In [47]:
model.classifier[1] = nn.Conv2d(in_channels=512,out_channels=5,kernel_size=1,stride =1)
for param in model.classifier[1].parameters():
  param.requires_grad = True

In [48]:
print(model)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)
critertion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)

cuda


In [50]:
def train_model(model, optimizer, train_loader, eval_loader, epochs=5):
    for epoch in range(epochs):
        model.train()
        train_correct = 0
        train_total = 0
        running_loss = 0.0

        print(f"\nEpoch {epoch+1}/{epochs}")

        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = critertion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)

            if (i + 1) % 10 == 0:
                batch_acc = (preds == labels).float().mean().item()
                print(f"[Batch {i+1}/{len(train_loader)}] "
                      f"Loss: {loss.item():.4f}, Batch Acc: {batch_acc:.4f}")


        train_acc = train_correct / train_total
        print(f"Epoch {epoch+1} Summary - "
              f"Loss: {running_loss:.4f}, "
              f"Train Accuracy: {train_acc:.4f}")


        model.eval()
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in eval_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = val_correct / val_total
        print(f"Validation Accuracy: {val_acc:.4f}")


In [ ]:
train_model(model,optimizer,train_loader,eval_loader)